Загружаем нужные библиотеки и таблицы

In [1]:
import pandas as pd
import psycopg2 as pg
import numpy as np

engine = pg.connect("dbname='db1' user='test_user' host='rc1a-p8bp15mmxsfwpbt0.mdb.yandexcloud.net' port='6432' password='j2M{CnnFq@'")
df_messages = pd.read_sql('select * from test.chat_messages', con=engine)
df_managers = pd.read_sql('select * from test.managers', con=engine)
df_leaders = pd.read_sql('select * from test.rops', con=engine)

C:\Users\andvi\AppData\Local\Temp\ipykernel_48940\2767668711.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_messages = pd.read_sql('select * from test.chat_messages', con=engine)
C:\Users\andvi\AppData\Local\Temp\ipykernel_48940\2767668711.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_managers = pd.read_sql('select * from test.managers', con=engine)
C:\Users\andvi\AppData\Local\Temp\ipykernel_48940\2767668711.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_leaders = pd.read_sql('select * from test.ro

Добавляем столбец с предыдушим типом сообщении

In [2]:
df_messages['prev_type'] = (
    df_messages.sort_values(by=['entity_id', 'created_at'])
    .groupby('entity_id')['type']
    .shift()
)

Фильтруем сообщения и оставим только первые сообщения каждого блока

In [3]:
conditions = (
    ((df_messages['type'] == 'incoming_chat_message') & 
     ((df_messages['prev_type'].isnull()) | (df_messages['prev_type'] != 'incoming_chat_message'))) |
    ((df_messages['type'] == 'outgoing_chat_message') & 
     ((df_messages['prev_type'].isnull()) | (df_messages['prev_type'] != 'outgoing_chat_message')))
)
first_messages = df_messages[conditions].copy()

Добавляем столбец чтобы обслеживать время получении сообщении от клиента

In [4]:
first_messages['prev_created_at'] = (
    first_messages.sort_values(by=['entity_id', 'created_at'])
    .groupby('entity_id')['created_at']
    .shift()
)

Создаем функцию чтобы считать время ответа в секундах

In [ ]:
def calculate_response_time(row):
    if pd.isnull(row['prev_created_at']):
        return np.nan
    
    # настроим столбца с датами чтобы работать с ними
    prev_created_at = pd.to_datetime(row['prev_created_at'], unit='s')
    created_at = pd.to_datetime(row['created_at'], unit='s')

    prev_time = prev_created_at.tz_localize('UTC').tz_convert('Europe/Moscow')
    cur_time = created_at.tz_localize('UTC').tz_convert('Europe/Moscow')
    
    # время получения сообщения от клиента в промежутке от 00:00:00 до 09:30:00
    if prev_time.time() < pd.Timestamp('09:30:00').time():
        start_of_day = pd.Timestamp(prev_time.date()).tz_localize('Europe/Moscow')
        seconds_after_midnight = (prev_time - start_of_day).total_seconds()
        adjustment = (9 * 3600 + 30 * 60) - seconds_after_midnight
        return (cur_time - prev_time).total_seconds() - adjustment
    
    # разница между дата получения сообщения и дата ответа >= 1
    elif (cur_time.date() - prev_time.date()).days >= 1:
        days_diff = (cur_time.date() - prev_time.date()).days
        return (cur_time - prev_time).total_seconds() - (days_diff * (9 * 3600 + 30 * 60))
    
    else:
        return (cur_time - prev_time).total_seconds()

Считаем время ответа менеджеров

In [8]:
first_messages['response_time'] = first_messages.apply(calculate_response_time, axis=1)
response_times = first_messages[
    (first_messages['type'] == 'outgoing_chat_message') &
    (first_messages['prev_type'].notnull())
].copy()

Собираем все нужные данные из остальных таблиц

In [10]:
df_managers['rop_id'] = df_managers['rop_id'].astype(int)
df_leaders['rop_id'] = df_leaders['rop_id'].astype(int)
df_merged = (
    response_times.merge(df_managers, left_on='created_by', right_on='mop_id')
    .merge(df_leaders, left_on='rop_id', right_on='rop_id')
)

считаем среднее время ответа менеджеров

In [11]:
df_result = (
    df_merged[df_merged['response_time'] > 0]
    .groupby(['mop_id', 'name_mop', 'rop_name'], as_index=False)
    .agg(avg_response_time=('response_time', 'mean'))
    .rename(columns={'mop_id': 'manager_id', 'name_mop': 'manager_name', 'rop_name': 'responsible_name'})
)

In [12]:
print(df_result)

    manager_id         manager_name responsible_name  avg_response_time
0      6645315       Гюнель и Илина         Катя РОП         516.302682
1      6744792         Юля и Наташа         Катя РОП         567.625296
2      6780177          Даша и Даша       Полина РОП         237.420000
3      7408305        Влада и Настя          Эля РОП         282.826087
4      7417617     Порхачева Полина          Эля РОП         502.214286
5      7996770           Лиза и Ева       Полина РОП         357.014577
6      8558094          Алина и Юля          Эля РОП         273.482609
7     10262485          Соня и Катя       Полина РОП         899.405204
8     10262493          Ками и Мила          Эля РОП         468.591111
9     10262497         Мария и Соня       Полина РОП         318.015464
10    10262513           Ира и Варя         Катя РОП        1199.651261
11    10465254         Настя и Даша         Катя РОП         211.225309
12    10465690       Настя и Малика          Эля РОП         335